## Main Model

In [1]:
import tensorflow as tf
import numpy as np
import pickle


In [2]:
def unpickle(file):
   
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [14]:
x_train=np.array([])
y_train=np.array([])
for i in range(1,6):
    
    batch = unpickle("data\cifar-10-batches-py\data_batch_%d"%(i))
    if len(x_train)==0 & len(y_train)==0:
        x_train = batch[b'data']
        y_train = batch[b'labels']
    else:
        x_train = np.concatenate((x_train, batch[b'data'])) 
        y_train = np.concatenate((y_train, batch[b'labels']))

In [30]:
test_batch = unpickle(r"data\cifar-10-batches-py\test_batch")
x_test = test_batch[b'data']
y_test = test_batch[b'labels']

## Model

In [31]:
def logit_block(layer,n_in,n_out,pkeep):
    #cnn_1
    W_conv1 = tf.get_variable('W_conv1', shape=[2, 2, n_in, n_out])
    b_conv1 = tf.get_variable('b_conv1', shape=[n_out])
    h_conv1 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(layer, W_conv1, strides=[1, 1, n_in, n_out], padding='SAME'), b_conv1))
    #dropout_1
    d_1=tf.nn.dropout(h_conv1,pkeep)
    #cnn_2
    W_conv2 = tf.get_variable('W_conv2', shape=[2, 2, n_out, n_out])
    b_conv2 = tf.get_variable('b_conv2', shape=[n_out])
    h_conv2 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_1, W_conv2, strides=[1, 1, n_out, n_out], padding='SAME'), b_conv2))
    #dropout_2
    d_2=tf.nn.dropout(h_conv2,pkeep)
    #fractional_max_pooling_
    return(tf.nn.fractional_max_pooling(d_2))
    
    

In [32]:
def compute_logits(x):
    x_image = tf.reshape(x,[-1,126,126,3])
    n1=320
    n2=640
    n3=960
    n4=1280
    n5=1600
    n6=1920
    n7=2240
    n8=2560
    
    #Block1
    #cnn_1
    W_conv1 = tf.get_variable('W_conv1', shape=[2, 2, 3, n1])
    b_conv1 = tf.get_variable('b_conv1', shape=[n1])
    h_conv1 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 3, n1], padding='SAME'), b_conv1))
    #cnn_2
    W_conv2 = tf.get_variable('W_conv2', shape=[2, 2, n1, n1])
    b_conv2 = tf.get_variable('b_conv2', shape=[n1])
    h_conv2 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv2, W_conv2, strides=[1, 1, n1, n1], padding='SAME'), b_conv2))
    #fractional_max_pooling_1
    fmp_1=tf.nn.fractional_max_pooling(h_conv2)
    
    block_1=logits_block(fmp_1,n1,n2,0.9)
    block_2=logits_nlock(block_1,n2,n3,0.8)
    block_3=logits_nlock(block_2,n3,n4,0.7)
    block_4=logits_nlock(block_3,n4,n5,0.6)
    block_5=logits_nlock(block_4,n5,n6,0.5)
    block_6=logits_nlock(block_5,n6,n7,0.5)
    block_7=logits_nlock(block_6,n7,n8,0.5)
    block_2=logits_nlock(block_7,n8,n8,0.5)
    
    #last_cnn
    W_conv = tf.get_variable('W_conv', shape=[2, 2, n8, n8])
    b_conv = tf.get_variable('b_conv', shape=[n8])
    h_conv = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv2, W_conv2, strides=[2, 2, n8, n8], padding='SAME'), b_conv))

    # fc layer to logits (2*2 )
    h_flat = tf.reshape(h_conv, [-1, 2*2*n8])
    W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n8, 10])
    b_fc1 = tf.get_variable('b_fc1', shape=[10])

    logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits')
    
    return(logits)

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy
    
    

In [33]:
dir_name = 'logs/scratch04x/summary'
with tf.Graph().as_default():

    x = tf.placeholder(tf.float32, [None, 126*126*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')

    
 
    with tf.name_scope('model'):
        logits = compute_logits(x_train)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y_train)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):

        opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(20001):
            if i%100 == 0: 
                (test_error, summary) = sess.run((accuracy,summary_op), {x:x_test, y:y_test})
                print("\rAfter step {0:3d}, test accuracy {1:0.4f}".format(i, test_error), flush=True)
                summary_writer_test.add_summary(summary, i)
    

ValueError: Dimension size must be evenly divisible by 47628 but is 153600000 for 'model/Reshape' (op: 'Reshape') with input shapes: [50000,3072], [4] and with input tensors computed as partial shapes: input[1] = [?,126,126,3].